# Goal
Jacobo de la Cuesta-Zuluaga.

After obtaining the relative abudnance of diverse genomes from the metagenomic samples of all available TUK samples, I will to perform descriptive statistics, determine the coabundance patterns of ca. M. alvus RL001 with other microbial taxa, and associate the abundance and presence of the microorganism with host health.

# Init

In [1]:
library(speedyseq)
library(UpSetR)
library(tidyverse)
library(cowplot)
library(LeyLabRMisc)
library(irr)
library(caret)

Loading required package: phyloseq

Attaching package: ‘speedyseq’

The following objects are masked from ‘package:phyloseq’:

    plot_bar, plot_heatmap, plot_tree, psmelt, tax_glom, tip_glom

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

********************************************************
Note: As of version 1.0.0, cowplot does not change the
  default ggplot2 theme anymore. To recover the previous
  behavior, execute:
  theme_set(theme_cowplot())
********************************************************

Loading required package: lpSolve
Loading required package: lattice

Attaching package: ‘lattice’

The following object is masked from ‘package:U

In [2]:
# Functions
# Extract phyloseq table as df
extract_phyloseq = function(physeq_obj){
    physeq_table = physeq_obj@`.Data`
    rownames(physeq_table) = NULL
    return(as.data.frame(physeq_table))
}

# Function to filter phyloseq object by list of taxa
pop_taxa = function(physeq, keep_Taxa){
  all_Taxa = taxa_names(physeq)
  all_Taxa <- all_Taxa[(all_Taxa %in% keep_Taxa)]
  return(prune_taxa(all_Taxa, physeq))
}

# Var

## Metadata

In [3]:
# Metadata
TUK_metadata_raw = "/ebio/abt3_projects/Anxiety_Twins_Metagenomes/data/metagenome/TUK-5projects/metadata/metadata-allAS.tsv" %>% 
    read_tsv() %>%
    filter(MG_dataset != "Visconti2019")
TUK_metadata_raw %>% glimpse

Parsed with column specification:
cols(
  Metagenome_run_accession = col_character(),
  i.IndividualID = col_character(),
  i.TwinID = col_character(),
  i.FamilyID = col_character(),
  i.DateOfBirth = col_character(),
  i.gender = col_character(),
  i.zygosity = col_character(),
  s.BMI = col_double(),
  E662_visit_date = col_character(),
  Q_sum = col_double(),
  AS = col_double(),
  Questionnaire = col_character(),
  MG_dataset = col_character(),
  s.FPBarcode = col_double()
)


Observations: 417
Variables: 14
$ Metagenome_run_accession <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …
$ i.IndividualID           <chr> "NA96922", "A13420", "NA64332", "NA65902", "…
$ i.TwinID                 <chr> "NA96922", "A3241", "NA64332", "NA65902", "N…
$ i.FamilyID               <chr> "NA9692", "A324", "NA6433", "NA6590", "NA659…
$ i.DateOfBirth            <chr> "1973-05-30", "1973-07-17", "1971-08-17", "1…
$ i.gender                 <chr> "F", "M", "F", "F", "F", "F", "F", "F", "F",…
$ i.zygosity               <chr> "MZ", "MZ", "DZ", "DZ", "DZ", "DZ", "DZ", "D…
$ s.BMI                    <dbl> 22.33987, 28.24785, 22.18279, 23.19296, 23.9…
$ E662_visit_date          <chr> "2005-01-20", "2010-03-15", "2005-01-20", "2…
$ Q_sum                    <dbl> 13, 8, 1, 4, 2, 7, 4, 18, 6, 5, 27, 16, 0, 1…
$ AS                       <dbl> 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1,…
$ Questionnaire            <chr> "Q10", "Q17A", "Q10", "Q17A", "Q17A", "Q17A"…
$ MG_dataset        

In [4]:
Visconti_data = "/ebio/abt3_projects/databases_no-backup/TUK/metagenome/Visconti2019/metadata_tidy-MG.tsv" %>%
    read_tsv()

Visconti_data %>% dfhead

Visconti_names = Visconti_data %>% 
    select(Metagenome_run_accession, i.TwinID, i.FamilyID, 
           i.DateOfBirth = BirthYear, i.gender = Sex, 
           i.zygosity = Zygosity, s.BMI = BMI, 
           i.IndividualID = Microbiome_ID, Sample) %>%
    mutate(i.DateOfBirth = as.character(i.DateOfBirth), 
           MG_dataset = "Visconti2019")

Visconti_names %>% dfhead

Parsed with column specification:
cols(
  .default = col_character(),
  BirthYear = col_double(),
  BMI = col_double(),
  Run = col_double(),
  Lane = col_double(),
  Total.Sequences = col_double(),
  C = col_double(),
  LR = col_double(),
  modelR = col_double(),
  LRstar = col_double(),
  diversity = col_double()
)
See spec(...) for full column specifications.


[1] 989  22


Metagenome_run_accession,kcl.mapped.i.TwinID,i.TwinID,i.FamilyID,BirthYear,Sex,Zygosity,BMI,Microbiome_ID,Sample,⋯,study_accession,instrument_model,library_layout,Total.Sequences,Read,C,LR,modelR,LRstar,diversity
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ERR3450607,A3301,A3301,A330,1948,F,MZ,44.67467,A3301,SAMEA5814677,⋯,PRJEB32731,Illumina HiSeq 2500,PAIRED,17577362,R1_final,0.9540563,2175409476,0.9806360,3443469349,14.51694
ERR3450608,A12221,A12221,A1222,1935,F,MZ,28.07811,A12221,SAMEA5814637,⋯,PRJEB32731,Illumina HiSeq 2500,PAIRED,15127605,R1_final,0.9517515,1874461536,0.9994476,953204765,17.40057
ERR3450609,NA,A8732,A873,1959,F,MZ,19.94806,A8732,SAMEA5814789,⋯,PRJEB32731,Illumina HiSeq 2500,PAIRED,9931280,R1_final,0.8941494,1230743805,0.9996583,2397424589,18.33600


[1] 989  10


Metagenome_run_accession,i.TwinID,i.FamilyID,i.DateOfBirth,i.gender,i.zygosity,s.BMI,i.IndividualID,Sample,MG_dataset
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
ERR3450607,A3301,A330,1948,F,MZ,44.67467,A3301,SAMEA5814677,Visconti2019
ERR3450608,A12221,A1222,1935,F,MZ,28.07811,A12221,SAMEA5814637,Visconti2019
ERR3450609,A8732,A873,1959,F,MZ,19.94806,A8732,SAMEA5814789,Visconti2019


In [5]:
TUK_metadata = full_join(Visconti_names, TUK_metadata_raw) %>%
    mutate(Sample = case_when(MG_dataset == "Visconti2019" ~ as.character(Sample),
                                               TRUE ~ str_c("x", s.FPBarcode))) %>%
    select(Sample, everything())

Joining, by = c("Metagenome_run_accession", "i.TwinID", "i.FamilyID", "i.DateOfBirth", "i.gender", "i.zygosity", "s.BMI", "i.IndividualID", "MG_dataset")


In [6]:
TUK_metadata %>% dfhead

[1] 1406   15


Sample,Metagenome_run_accession,i.TwinID,i.FamilyID,i.DateOfBirth,i.gender,i.zygosity,s.BMI,i.IndividualID,MG_dataset,E662_visit_date,Q_sum,AS,Questionnaire,s.FPBarcode
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>
SAMEA5814677,ERR3450607,A3301,A330,1948,F,MZ,44.67467,A3301,Visconti2019,NA,NA,NA,NA,NA
SAMEA5814637,ERR3450608,A12221,A1222,1935,F,MZ,28.07811,A12221,Visconti2019,NA,NA,NA,NA,NA
SAMEA5814789,ERR3450609,A8732,A873,1959,F,MZ,19.94806,A8732,Visconti2019,NA,NA,NA,NA,NA


## Taxonomic profiles

In [7]:
TUK_deep = "/ebio/abt3_projects/vadinCA11/data/V11/heritability/TUK_deep_profiles"

In [8]:
# Bracken files
Visconti_Bracken_Raw = file.path(TUK_deep, "Visconti", "kraken", "all-combined-bracken.tsv") %>%
    read_tsv()

Visconti_Bracken_Raw %>% dfhead

Parsed with column specification:
cols(
  .default = col_double(),
  name = col_character(),
  taxonomy_lvl = col_character(),
  taxonomy = col_character(),
  taxIDs = col_character()
)
See spec(...) for full column specifications.


[1] 15679  2011


name,taxonomy_id,taxonomy_lvl,taxonomy,taxIDs,SAMEA5814677_num,SAMEA5814677_frac,SAMEA5814637_num,SAMEA5814637_frac,SAMEA5814789_num,⋯,SAMEA5814845_num,SAMEA5814845_frac,SAMEA5814186_num,SAMEA5814186_frac,SAMEA5814401_num,SAMEA5814401_frac,SAMEA5814690_num,SAMEA5814690_frac,SAMEA5814506_num,SAMEA5814506_frac
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pedobacter sp. AJM,2003629,S,Bacteria;Bacteroidetes;Sphingobacteriia;Sphingobacteriales;Sphingobacteriaceae;Pedobacter;Pedobacter sp. AJM,131567;2;1783270;68336;976;117747;200666;84566;84567;2003629,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Maribacter sp. 1_2014MBL_MicDiv,1644130,S,Bacteria;Bacteroidetes;Flavobacteriia;Flavobacteriales;Flavobacteriaceae;Maribacter;Maribacter sp. 1_2014MBL_MicDiv,131567;2;1783270;68336;976;117743;200644;49546;252356;1644130,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Selenomonas bovis,416586,S,Bacteria;Firmicutes;Negativicutes;Selenomonadales;Selenomonadaceae;Selenomonas;Selenomonas bovis,131567;2;1783272;1239;909932;909929;1843491;970;416586,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [9]:
# Bracken files
vadin_Bracken_Raw = file.path(TUK_deep, "vadinCA11", "kraken", "all-combined-bracken.tsv") %>%
    read_tsv()

vadin_Bracken_Raw %>% dfhead

Parsed with column specification:
cols(
  .default = col_double(),
  name = col_character(),
  taxonomy_lvl = col_character(),
  taxonomy = col_character(),
  taxIDs = col_character()
)
See spec(...) for full column specifications.


[1] 11722   301


name,taxonomy_id,taxonomy_lvl,taxonomy,taxIDs,1006109_num,1006109_frac,1006179_num,1006179_frac,1006171_num,⋯,1020132_num,1020132_frac,1020146_num,1020146_frac,1020174_num,1020174_frac,mock_num,mock_frac,water_num,water_frac
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pedobacter sp. AJM,2003629,S,Bacteria;Bacteroidetes;Sphingobacteriia;Sphingobacteriales;Sphingobacteriaceae;Pedobacter;Pedobacter sp. AJM,131567;2;1783270;68336;976;117747;200666;84566;84567;2003629,0,0e+00,0,0e+00,0,⋯,0,0,0,0,0,0,0,0,0,0
Maribacter sp. 1_2014MBL_MicDiv,1644130,S,Bacteria;Bacteroidetes;Flavobacteriia;Flavobacteriales;Flavobacteriaceae;Maribacter;Maribacter sp. 1_2014MBL_MicDiv,131567;2;1783270;68336;976;117743;200644;49546;252356;1644130,0,0e+00,0,0e+00,989,⋯,0,0,0,0,0,0,0,0,0,0
Selenomonas bovis,416586,S,Bacteria;Firmicutes;Negativicutes;Selenomonadales;Selenomonadaceae;Selenomonas;Selenomonas bovis,131567;2;1783272;1239;909932;909929;1843491;970;416586,329,1e-05,207,1e-05,361,⋯,0,0,0,0,0,0,0,0,0,0


In [10]:
# Bracken files
BGI250_Bracken_Raw = file.path(TUK_deep, "BGI250", "kraken", "all-combined-bracken.tsv") %>%
    read_tsv()

BGI250_Bracken_Raw %>% dfhead

Parsed with column specification:
cols(
  .default = col_double(),
  name = col_character(),
  taxonomy_lvl = col_character(),
  taxonomy = col_character(),
  taxIDs = col_character()
)
See spec(...) for full column specifications.


[1] 15282   505


name,taxonomy_id,taxonomy_lvl,taxonomy,taxIDs,YSZC12003_35365_num,YSZC12003_35365_frac,YSZC12003_35366_num,YSZC12003_35366_frac,YSZC12003_35387_num,⋯,YSZC12003_37830_num,YSZC12003_37830_frac,YSZC12003_37877_num,YSZC12003_37877_frac,YSZC12003_37878_num,YSZC12003_37878_frac,YSZC12003_37879_num,YSZC12003_37879_frac,YSZC12003_37880_num,YSZC12003_37880_frac
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pedobacter sp. AJM,2003629,S,Bacteria;Bacteroidetes;Sphingobacteriia;Sphingobacteriales;Sphingobacteriaceae;Pedobacter;Pedobacter sp. AJM,131567;2;1783270;68336;976;117747;200666;84566;84567;2003629,0,0,0,0,222,⋯,0,0,0,0,0,0,0,0,0,0
Maribacter sp. 1_2014MBL_MicDiv,1644130,S,Bacteria;Bacteroidetes;Flavobacteriia;Flavobacteriales;Flavobacteriaceae;Maribacter;Maribacter sp. 1_2014MBL_MicDiv,131567;2;1783270;68336;976;117743;200644;49546;252356;1644130,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Streptomyces ochraceiscleroticus,47761,S,Bacteria;Actinobacteria;Actinobacteria;Streptomycetales;Streptomycetaceae;Streptomyces;Streptomyces ochraceiscleroticus,131567;2;1783272;201174;1760;85011;2062;1883;47761,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [11]:
# Bracken files
anx_Bracken_Raw = file.path(TUK_deep, "anx", "kraken", "all-combined-bracken.tsv") %>%
    read_tsv()

anx_Bracken_Raw %>% dfhead

Parsed with column specification:
cols(
  .default = col_double(),
  name = col_character(),
  taxonomy_lvl = col_character(),
  taxonomy = col_character(),
  taxIDs = col_character()
)
See spec(...) for full column specifications.


[1] 14828   357


name,taxonomy_id,taxonomy_lvl,taxonomy,taxIDs,1003127_num,1003127_frac,1003130_num,1003130_frac,1003182_num,⋯,1002409_num,1002409_frac,1002474_num,1002474_frac,1002774_num,1002774_frac,1002868_num,1002868_frac,1003466_num,1003466_frac
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pedobacter sp. AJM,2003629,S,Bacteria;Bacteroidetes;Sphingobacteriia;Sphingobacteriales;Sphingobacteriaceae;Pedobacter;Pedobacter sp. AJM,131567;2;1783270;68336;976;117747;200666;84566;84567;2003629,0,0,135,0e+00,0,⋯,0,0e+00,0,0,0,0,0,0,0,0
Bradyrhizobium sp. Ghvi,1855319,S,Bacteria;Proteobacteria;Alphaproteobacteria;Rhizobiales;Bradyrhizobiaceae;Bradyrhizobium;Bradyrhizobium sp. Ghvi,131567;2;1224;28211;356;41294;374;1855319,0,0,0,0e+00,0,⋯,0,0e+00,0,0,0,0,0,0,0,0
Maribacter sp. 1_2014MBL_MicDiv,1644130,S,Bacteria;Bacteroidetes;Flavobacteriia;Flavobacteriales;Flavobacteriaceae;Maribacter;Maribacter sp. 1_2014MBL_MicDiv,131567;2;1783270;68336;976;117743;200644;49546;252356;1644130,0,0,278,1e-05,0,⋯,220,1e-05,0,0,0,0,0,0,0,0


In [12]:
header_vars = c('name', 'taxonomy_id', 'taxonomy_lvl', 'taxonomy', 'taxIDs')

In [13]:
combined_bracken_raw = left_join(Visconti_Bracken_Raw, BGI250_Bracken_Raw, by = header_vars) %>%
    left_join(., vadin_Bracken_Raw, by = header_vars) %>% 
    left_join(., anx_Bracken_Raw, by = header_vars) %>% 
    select(-matches("\\.y"))

colnames(combined_bracken_raw) = colnames(combined_bracken_raw) %>% str_remove_all("\\.x") 

combined_bracken_raw %>% dfhead

[1] 15679  3123


name,taxonomy_id,taxonomy_lvl,taxonomy,taxIDs,SAMEA5814677_num,SAMEA5814677_frac,SAMEA5814637_num,SAMEA5814637_frac,SAMEA5814789_num,⋯,1002409_num,1002409_frac,1002474_num,1002474_frac,1002774_num,1002774_frac,1002868_num,1002868_frac,1003466_num,1003466_frac
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pedobacter sp. AJM,2003629,S,Bacteria;Bacteroidetes;Sphingobacteriia;Sphingobacteriales;Sphingobacteriaceae;Pedobacter;Pedobacter sp. AJM,131567;2;1783270;68336;976;117747;200666;84566;84567;2003629,0,0,0,0,0,⋯,0,0e+00,0,0e+00,0,0e+00,0,0e+00,0,0
Maribacter sp. 1_2014MBL_MicDiv,1644130,S,Bacteria;Bacteroidetes;Flavobacteriia;Flavobacteriales;Flavobacteriaceae;Maribacter;Maribacter sp. 1_2014MBL_MicDiv,131567;2;1783270;68336;976;117743;200644;49546;252356;1644130,0,0,0,0,0,⋯,220,1e-05,0,0e+00,0,0e+00,0,0e+00,0,0
Selenomonas bovis,416586,S,Bacteria;Firmicutes;Negativicutes;Selenomonadales;Selenomonadaceae;Selenomonas;Selenomonas bovis,131567;2;1783272;1239;909932;909929;1843491;970;416586,0,0,0,0,0,⋯,0,0e+00,515,2e-05,396,2e-05,296,1e-05,0,0


In [14]:
# Clean table
tax_levs = c('Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species')
combined_bracken = combined_bracken_raw %>% 
    select(-taxonomy_lvl, -taxIDs) %>%
    separate(taxonomy, into = tax_levs, sep = ";") %>% 
    mutate(taxonomy_id = str_c("TID", taxonomy_id))

combined_bracken %>% dfhead

[1] 15679  3127


name,taxonomy_id,Domain,Phylum,Class,Order,Family,Genus,Species,SAMEA5814677_num,⋯,1002409_num,1002409_frac,1002474_num,1002474_frac,1002774_num,1002774_frac,1002868_num,1002868_frac,1003466_num,1003466_frac
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pedobacter sp. AJM,TID2003629,Bacteria,Bacteroidetes,Sphingobacteriia,Sphingobacteriales,Sphingobacteriaceae,Pedobacter,Pedobacter sp. AJM,0,⋯,0,0e+00,0,0e+00,0,0e+00,0,0e+00,0,0
Maribacter sp. 1_2014MBL_MicDiv,TID1644130,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Maribacter,Maribacter sp. 1_2014MBL_MicDiv,0,⋯,220,1e-05,0,0e+00,0,0e+00,0,0e+00,0,0
Selenomonas bovis,TID416586,Bacteria,Firmicutes,Negativicutes,Selenomonadales,Selenomonadaceae,Selenomonas,Selenomonas bovis,0,⋯,0,0e+00,515,2e-05,396,2e-05,296,1e-05,0,0


In [15]:
# Separate into relabund and absolute counts tables
# Fix col names

# Counts
TUK_counts = combined_bracken %>% 
    select(-ends_with("_frac"))

colnames(TUK_counts) = colnames(TUK_counts) %>% 
    str_remove_all("_num") %>%
    purrr::map_chr(function(name) if_else(str_detect(name, "^[0-9]"), 
                                          str_c("x", name), name)) 
# Relabund    
TUK_relabund = combined_bracken %>% 
    select(-ends_with("_num"))

colnames(TUK_relabund) = colnames(TUK_relabund) %>% 
    str_remove_all("_frac") %>%
    purrr::map_chr(function(name) if_else(str_detect(name, "^[0-9]"), 
                                          str_c("x", name), name)) 

In [16]:
TUK_relabund %>% dfhead
TUK_counts %>% dfhead

[1] 15679  1568


name,taxonomy_id,Domain,Phylum,Class,Order,Family,Genus,Species,SAMEA5814677,⋯,x1014133,x1014134,x1014285,x1014291,x1002424,x1002409,x1002474,x1002774,x1002868,x1003466
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pedobacter sp. AJM,TID2003629,Bacteria,Bacteroidetes,Sphingobacteriia,Sphingobacteriales,Sphingobacteriaceae,Pedobacter,Pedobacter sp. AJM,0,⋯,0,0,0e+00,0,0e+00,0e+00,0e+00,0e+00,0e+00,0
Maribacter sp. 1_2014MBL_MicDiv,TID1644130,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Maribacter,Maribacter sp. 1_2014MBL_MicDiv,0,⋯,0,0,1e-05,0,0e+00,1e-05,0e+00,0e+00,0e+00,0
Selenomonas bovis,TID416586,Bacteria,Firmicutes,Negativicutes,Selenomonadales,Selenomonadaceae,Selenomonas,Selenomonas bovis,0,⋯,0,0,0e+00,0,1e-05,0e+00,2e-05,2e-05,1e-05,0


[1] 15679  1568


name,taxonomy_id,Domain,Phylum,Class,Order,Family,Genus,Species,SAMEA5814677,⋯,x1014133,x1014134,x1014285,x1014291,x1002424,x1002409,x1002474,x1002774,x1002868,x1003466
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pedobacter sp. AJM,TID2003629,Bacteria,Bacteroidetes,Sphingobacteriia,Sphingobacteriales,Sphingobacteriaceae,Pedobacter,Pedobacter sp. AJM,0,⋯,0,0,0,0,0,0,0,0,0,0
Maribacter sp. 1_2014MBL_MicDiv,TID1644130,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Maribacter,Maribacter sp. 1_2014MBL_MicDiv,0,⋯,0,0,205,0,0,220,0,0,0,0
Selenomonas bovis,TID416586,Bacteria,Firmicutes,Negativicutes,Selenomonadales,Selenomonadaceae,Selenomonas,Selenomonas bovis,0,⋯,0,0,0,0,348,0,515,396,296,0


# Phyloseq object

In [17]:
# Create metadata table selecting one entry per twin

retained_variables = c("Sample", "Metagenome_run_accession", "i.TwinID", 
                       "i.FamilyID", "i.DateOfBirth", "i.gender", "i.zygosity", 
                       "s.BMI", "i.IndividualID", "MG_dataset")


meta_phyloseq = TUK_metadata %>%
    select(retained_variables) %>%
    filter(MG_dataset != "BGI250") %>%
    group_by(i.TwinID) %>%
    slice(1) %>%
    ungroup() %>% 
    mutate_if(is.character, as.factor) %>%
    column_to_rownames("Sample")

meta_phyloseq %>% dfhead

[1] 1036    9


,Metagenome_run_accession,i.TwinID,i.FamilyID,i.DateOfBirth,i.gender,i.zygosity,s.BMI,i.IndividualID,MG_dataset
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>
SAMEA5814089,ERR3450857,A1001,A100,1947,M,DZ,28.14091,A1001,Visconti2019
SAMEA5814088,ERR3450926,A1002,A100,1947,M,DZ,30.52770,A1002,Visconti2019
SAMEA5814063,ERR3450938,A10041,A1004,1955,F,DZ,23.33768,A10041,Visconti2019


In [18]:
# Create tax table and conver empty fields to NA
tax_phyloseq = TUK_counts %>%
    select(taxonomy_id, tax_levs) %>%
    mutate_all(., list(~na_if(.,""))) %>%
    column_to_rownames("taxonomy_id") %>%
    as.matrix()

tax_phyloseq %>% dfhead

[1] 15679     7


,Domain,Phylum,Class,Order,Family,Genus,Species
TID2003629,Bacteria,Bacteroidetes,Sphingobacteriia,Sphingobacteriales,Sphingobacteriaceae,Pedobacter,Pedobacter sp. AJM
TID1644130,Bacteria,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Maribacter,Maribacter sp. 1_2014MBL_MicDiv
TID416586,Bacteria,Firmicutes,Negativicutes,Selenomonadales,Selenomonadaceae,Selenomonas,Selenomonas bovis


In [19]:
# Create OTU table and convert NAs to 0

samples_retained = intersect(rownames(meta_phyloseq), colnames(TUK_counts))

count_phyloseq = TUK_counts %>%
    select(taxonomy_id, samples_retained) %>%
    column_to_rownames("taxonomy_id")

count_phyloseq[is.na(count_phyloseq)] = 0

count_phyloseq = as.matrix(count_phyloseq)

count_phyloseq %>% dfhead

[1] 15679  1014


,SAMEA5814089,SAMEA5814088,SAMEA5814063,SAMEA5814064,SAMEA5814071,SAMEA5814072,SAMEA5814076,SAMEA5814083,SAMEA5814097,SAMEA5814098,⋯,SAMEA5814839,SAMEA5814840,SAMEA5814842,SAMEA5814845,x1014033,SAMEA5814846,SAMEA5814847,SAMEA5814848,SAMEA5814849,SAMEA5814659
TID2003629,0,0,0,0,0,0,0,0,0,0,⋯,0,159,0,0,0,0,0,0,0,0
TID1644130,0,0,0,0,0,0,166,0,0,170,⋯,0,0,0,0,0,0,0,0,0,0
TID416586,0,0,0,288,0,0,0,0,242,0,⋯,0,0,0,0,0,0,224,0,0,0


In [20]:
# Creating phyloseq object
OTU = otu_table(count_phyloseq, taxa_are_rows = T)
TAX = tax_table(tax_phyloseq)
META = sample_data(meta_phyloseq)

In [21]:
TUK_physeq_count = phyloseq(OTU, TAX, META)

In [22]:
TUK_physeq_count

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 15679 taxa and 1014 samples ]
sample_data() Sample Data:       [ 1014 samples by 9 sample variables ]
tax_table()   Taxonomy Table:    [ 15679 taxa by 7 taxonomic ranks ]

# Retain informative samples
I will only use samples for which family and zygosity information is available

In [23]:
# Metadata table
metadata_heritability = TUK_physeq_count %>% 
    phyloseq2df(., sample_data)


# Add number of sequences to metadata
sequence_count = count_phyloseq %>%
    as.data.frame() %>%
    summarise_all(sum) %>%
    gather(Sample, Sequences)

metadata_heritability = left_join(metadata_heritability, sequence_count, by = "Sample") %>%
    arrange(Sample)

metadata_heritability %>% dfhead

[1] 1014   11


Metagenome_run_accession,i.TwinID,i.FamilyID,i.DateOfBirth,i.gender,i.zygosity,s.BMI,i.IndividualID,MG_dataset,Sample,Sequences
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<chr>,<dbl>
ERR3451335,NA61,NA6,1933,F,MZ,21.00215,NA61,Visconti2019,SAMEA5813845,11373236
ERR3450671,NA62,NA6,1933,F,MZ,27.53016,NA62,Visconti2019,SAMEA5813846,9344991
ERR3450655,A8951,A895,1931,F,MZ,31.74910,A8951,Visconti2019,SAMEA5813847,13038333


In [24]:
# Retain samples with >12 million reads for sensitivity analysis

seq_depth = 10 * 1e6

deep_samples = metadata_heritability %>%
    filter(Sequences >= seq_depth) %>%
    pull(Sample)

TUK_physeq_deep = prune_samples(deep_samples, TUK_physeq_count)
TUK_physeq_deep

# Filter metadata to retained samples
filt_metadata_heritability = metadata_heritability %>%
    filter(Sample %in% deep_samples) %>%
    arrange(Sample)

filt_metadata_heritability %>% dfhead

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 15679 taxa and 723 samples ]
sample_data() Sample Data:       [ 723 samples by 9 sample variables ]
tax_table()   Taxonomy Table:    [ 15679 taxa by 7 taxonomic ranks ]

[1] 723  11


Metagenome_run_accession,i.TwinID,i.FamilyID,i.DateOfBirth,i.gender,i.zygosity,s.BMI,i.IndividualID,MG_dataset,Sample,Sequences
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<chr>,<dbl>
ERR3451335,NA61,NA6,1933,F,MZ,21.00215,NA61,Visconti2019,SAMEA5813845,11373236
ERR3450655,A8951,A895,1931,F,MZ,31.74910,A8951,Visconti2019,SAMEA5813847,13038333
ERR3451002,A21,A2,1935,F,MZ,27.49109,A21,Visconti2019,SAMEA5813848,16477545


In [25]:
# Identify subjects with family and zygosity information
# Families with 2 subjects
# TUK_physeq_count
complete_families = TUK_physeq_deep %>%
    phyloseq2df(., sample_data) %>%
    count(i.FamilyID) %>% 
    filter(n == 2) %>%
    pull(i.FamilyID)

# Filter metadata table to retain only these subjects
retain_samples = TUK_physeq_deep %>% 
    phyloseq2df(., sample_data) %>%
    filter(i.FamilyID %in% complete_families) %>%
    pull(Sample)

# Update metadata heritability
filt_metadata_heritability = filt_metadata_heritability %>%
    filter(Sample %in% retain_samples) %>%
    arrange(Sample)

filt_metadata_heritability %>% dfhead

[1] 394  11


Metagenome_run_accession,i.TwinID,i.FamilyID,i.DateOfBirth,i.gender,i.zygosity,s.BMI,i.IndividualID,MG_dataset,Sample,Sequences
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<chr>,<dbl>
ERR3451002,A21,A2,1935,F,MZ,27.49109,A21,Visconti2019,SAMEA5813848,16477545
ERR3451517,A22,A2,1935,F,MZ,28.89664,A22,Visconti2019,SAMEA5813849,18679325
ERR3451289,A3861,A386,1925,F,DZ,20.64814,A3861,Visconti2019,SAMEA5813853,14838823


In [26]:
# Prune from phyloseq object
TUK_count_zygosity = prune_samples(retain_samples, TUK_physeq_deep)
TUK_count_zygosity

# Create relative abundance object from count data
TUK_relabund_zygosity  = transform_sample_counts(TUK_count_zygosity, function(x) x / sum(x))
TUK_relabund_zygosity

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 15679 taxa and 394 samples ]
sample_data() Sample Data:       [ 394 samples by 9 sample variables ]
tax_table()   Taxonomy Table:    [ 15679 taxa by 7 taxonomic ranks ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 15679 taxa and 394 samples ]
sample_data() Sample Data:       [ 394 samples by 9 sample variables ]
tax_table()   Taxonomy Table:    [ 15679 taxa by 7 taxonomic ranks ]

# Prepare data for ICC analyses

## Filter species table by prevalence
Methods for estimating heritability assume normally distributed data, however, transforming taxa counts may be a problem due to sparcity. Therefore, I will agglomerate taxa to the family and genus levels.

In [27]:
# Filter by prevalence and abundance
# Note that I use the relabund table to determine taxa to remove
# But remove them from absolut count table
TUK_relabund_genus = tax_glom(TUK_relabund_zygosity, taxrank = "Genus")

TUK_count_genus = tax_glom(TUK_count_zygosity, taxrank = 'Genus')

TUK_count_genus

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2182 taxa and 394 samples ]
sample_data() Sample Data:       [ 394 samples by 9 sample variables ]
tax_table()   Taxonomy Table:    [ 2182 taxa by 7 taxonomic ranks ]

In [28]:
# Filter taxa by abundance and prevalence
prev_cutoff = 0.05
abund_cutoff = (0.0005/100)
taxa_filtered = TUK_relabund_genus %>%
    filter_taxa(physeq = ., function(x) sum(x > 0) > (prev_cutoff*length(x)), T) %>%
    filter_taxa(physeq = ., function(x) mean(x) > abund_cutoff, T) %>%
    taxa_names
                
TUK_glom = pop_taxa(TUK_count_genus, taxa_filtered)
TUK_glom   

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 392 taxa and 394 samples ]
sample_data() Sample Data:       [ 394 samples by 9 sample variables ]
tax_table()   Taxonomy Table:    [ 392 taxa by 7 taxonomic ranks ]

In [29]:
TUK_glom %>% 
    phyloseq2df(tax_table) %>% 
    filter(Domain == "Archaea")

Domain,Phylum,Class,Order,Family,Genus,Species,OTU
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Archaea,Euryarchaeota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobacterium,NA,TID1915424
Archaea,Euryarchaeota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter,NA,TID2173
Archaea,Euryarchaeota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanosphaera,NA,TID2317
Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomassiliicoccaceae,Methanomassiliicoccus,NA,TID1406512


##  Transform counts and obtain residuals
I will use Yeo-Johnson transformations on sequence counts

In [30]:
# Obtain OTU table
filter_glom = TUK_glom %>% 
    phyloseq2df(otu_table) %>%
    column_to_rownames("OTU") %>%
    t() %>%
    as.data.frame()
filter_glom %>% dfhead
# Use the Yeo-Johnson transformation
YJ_lambda_glom = preProcess(filter_glom, method = c("YeoJohnson")) 
YJ_lambda_glom

# # Obtained transformed values data frame
YJ_glom = YJ_lambda_glom %>% 
    predict(filter_glom) %>%
    as.data.frame

YJ_glom %>% dfhead

[1] 394 392


,TID1915424,TID2173,TID2317,TID1406512,TID712118,TID1660,TID419015,TID216816,TID78258,TID230143,⋯,TID1930011,TID1197717,TID651822,TID638849,TID457415,TID1262686,TID216944,TID1262904,TID1262865,TID239935
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SAMEA5814063,0,35340,0,0,3563,0,0,7466,0,0,⋯,3381,0,0,0,0,0,0,0,314,143294
SAMEA5814064,0,155357,0,0,3939,463,0,628957,0,0,⋯,2463,10091,129,455,1558,1806,0,27958,303,970
SAMEA5814071,0,129514,0,0,7420,637,0,11978,0,0,⋯,1706,186,0,147,163,917,0,21705,0,279


Created from 394 samples and 392 variables

Pre-processing:
  - ignored (0)
  - Yeo-Johnson transformation (392)

Lambda estimates for Yeo-Johnson transformation:
    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-2.58619 -0.51925 -0.06689 -0.20983  0.15621  0.54988 


[1] 394 392


,TID1915424,TID2173,TID2317,TID1406512,TID712118,TID1660,TID419015,TID216816,TID78258,TID230143,⋯,TID1930011,TID1197717,TID651822,TID638849,TID457415,TID1262686,TID216944,TID1262904,TID1262865,TID239935
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SAMEA5814063,0,14.54381,0,0,23.45640,0.00000,0,20.00897,0,0,⋯,13.52464,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.00000,0.9629543,35.760821
SAMEA5814064,0,17.43410,0,0,24.09146,13.47540,0,47.52835,0,0,⋯,12.72472,3.700178,2.185771,6.262625,4.208526,6.166567,0,11.10758,0.9628607,12.580451
SAMEA5814071,0,17.06506,0,0,28.44753,14.82764,0,22.09287,0,0,⋯,11.83370,2.973544,0.000000,5.090308,3.417637,5.706520,0,10.81086,0.0000000,9.172445


In [31]:
# # Metadata table
# metadata_heritability = TUK_glom %>% 
#     phyloseq2df(sample_data)

# # Add number of sequences to metadata
# sequence_count = count_phyloseq %>%
#     as.data.frame() %>%
#     summarise_all(sum) %>%
#     gather(Sample, Sequences)

# metadata_heritability = left_join(metadata_heritability, sequence_count, by = "Sample") %>%
#     arrange(Sample)

# metadata_heritability %>% dfhead

In [32]:
# Oganize table of YJ transformed residuals after adjusting for BMI, sex and sequencing depth
YJ_glom = YJ_glom %>% 
    rownames_to_column("Sample") %>%
    arrange(Sample) %>%
    column_to_rownames("Sample")



# # Regression of transformed
tax_residuals = function(abund){
    tax_lm = lm(abund ~ s.BMI + Sequences + i.gender, 
                data = filt_metadata_heritability)
    
    tax_res = tax_lm$residuals
    tax_res
}

residuals_glom = YJ_glom %>% 
    map_dfc(tax_residuals) %>%
    as.data.frame

rownames(residuals_glom) = filt_metadata_heritability$Sample
residuals_glom %>% dfhead

[1] 394 392


,TID1915424,TID2173,TID2317,TID1406512,TID712118,TID1660,TID419015,TID216816,TID78258,TID230143,⋯,TID1930011,TID1197717,TID651822,TID638849,TID457415,TID1262686,TID216944,TID1262904,TID1262865,TID239935
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SAMEA5813848,-0.2950344,-2.881155,-0.1502566,-0.05179938,8.628771,5.634507,1.0366845,6.1888523,0.9377414,1.6757326,⋯,1.0112078,-1.245693,1.5817652,2.110917,1.774803,1.974126,-0.6633023,2.0627573,-0.1527645,12.817942
SAMEA5813849,-0.3119669,6.936152,-0.1348863,-0.05173814,17.647129,4.923341,1.0436586,0.8228964,-0.3281974,1.4864006,⋯,3.3124786,-1.281841,-0.7787841,1.937544,-1.588082,1.915303,1.3909453,1.4717251,-0.1625294,-21.231411
SAMEA5813853,-0.2906406,10.394014,0.8113238,0.50213542,11.080513,12.703062,-0.2847827,-3.5418497,-0.2658950,-0.7824149,⋯,0.5998401,2.083410,1.3125682,3.081103,2.467857,1.340892,-0.6136029,-0.2788522,-0.1768986,2.784246


# Organize twin IDs

In [33]:
# Assign twin number per family
ICC_metadata_heritability = filt_metadata_heritability %>% 
    arrange(i.FamilyID) %>%
    mutate(Twin_n = rep(1:2, nrow(.)/2))

ICC_metadata_heritability %>% dfhead

# Number of MZ and DZ pairs
ICC_metadata_heritability %>% group_by(i.zygosity) %>% summarise(n = n()/2) 

[1] 394  12


Metagenome_run_accession,i.TwinID,i.FamilyID,i.DateOfBirth,i.gender,i.zygosity,s.BMI,i.IndividualID,MG_dataset,Sample,Sequences,Twin_n
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<chr>,<dbl>,<int>
ERR3450938,A10041,A1004,1955,F,DZ,23.33768,A10041,Visconti2019,SAMEA5814063,13603936,1
ERR3451366,A10042,A1004,1955,F,DZ,25.91068,A10042,Visconti2019,SAMEA5814064,11995247,2
ERR3451199,A10071,A1007,1951,F,DZ,26.58076,A10071,Visconti2019,SAMEA5814071,13525719,1


i.zygosity,n
<fct>,<dbl>
DZ,117
MZ,80


# ICC tests

In [34]:
# residuals glom
icc_residuals = function(tax_residuals, metadata_table){
    
    # Determine IDs of MZ and DZ twins
    # Pull MZ ones and twos separately
    MZ_ones = metadata_table %>% 
        filter(i.zygosity == "MZ" & Twin_n == 1) %>% 
        pull(Sample)
    MZ_twos = metadata_table %>% 
        filter(i.zygosity == "MZ" & Twin_n == 2) %>% 
        pull(Sample)
    
    # Pull DZ ones and twos separately
    DZ_ones = metadata_table %>% 
        filter(i.zygosity == "DZ" & Twin_n == 1) %>% 
        pull(Sample)
    
    DZ_twos = metadata_table %>% 
        filter(i.zygosity == "DZ" & Twin_n == 2) %>% 
        pull(Sample)
        
    residuals_family = tax_residuals %>% 
        rownames_to_column("Sample")
        
    # ICC in MZ
    residuals_MZ_ones = residuals_family %>% 
        filter(Sample %in% MZ_ones) %>%
        select(-Sample)
    
    residuals_MZ_twos = residuals_family %>% 
        filter(Sample %in% MZ_twos) %>%
        select(-Sample)

    icc_MZ = map2(residuals_MZ_ones, residuals_MZ_twos, 
               function(x,y) cbind(x,y) %>% icc) 
                  
    # ICC in DZ
    residuals_DZ_ones = residuals_family %>% 
        filter(Sample %in% DZ_ones) %>%
        select(-Sample)
    
    residuals_DZ_twos = residuals_family %>% 
        filter(Sample %in% DZ_twos) %>%
        select(-Sample)

    icc_DZ = map2(residuals_DZ_ones, residuals_DZ_twos, 
               function(x,y) cbind(x,y) %>% icc) 

    # Create data frame with ICC and p-values
    icc_df = map2_dfc(icc_MZ, icc_DZ, 
             function(x,y) c(x$value, x$p.value, y$value, y$p.value))
    icc_df = icc_df %>% 
        t %>% 
        as.data.frame %>%
        rownames_to_column
                      
    colnames(icc_df) = c("OTU", "r_MZ", "p_MZ", "r_DZ", "p_DZ")
    # Difference between ICCs
    icc_df = icc_df %>% mutate(Diff = r_MZ - r_DZ)
    icc_df

}

In [35]:
# Obtain ICC table
icc_table_raw = icc_residuals(residuals_glom, ICC_metadata_heritability)

In [36]:
# add taxonomy to ICC table
icc_table = TUK_glom %>% 
    phyloseq2df(tax_table) %>%
    select(-c(Species)) %>%
    left_join(., icc_table_raw, by = "OTU")

# Add adjusted p-values
icc_table = icc_table %>%
    mutate(pad_MZ = p.adjust(p_MZ, method = "BH"), pad_DZ = p.adjust(p_DZ, method = "BH"))

Archaea_icc = icc_table %>% 
    filter(Order == "Methanomassiliicoccales") %>%
    select(Family, Genus, r_MZ, pad_MZ, r_DZ, pad_DZ, Diff) %>%
    arrange(-Diff)

Archaea_icc %>% mutate_if(is.numeric, function(x) round(x, 3))

Family,Genus,r_MZ,pad_MZ,r_DZ,pad_DZ,Diff
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Methanomassiliicoccaceae,Methanomassiliicoccus,0.094,0.375,0.061,0.962,0.033


In [46]:
selected_taxa = c("Faecalibacterium",  "Methanobrevibacter", 
                  "Bifidobacterium", "Christensenella", "Blautia")

icc_table %>% 
    filter(Genus %in% selected_taxa) %>% 
    filter(Diff > 0 & r_MZ > 0 & pad_MZ < 0.2) %>%
    select(Family, Genus, r_MZ, pad_MZ, r_DZ, pad_DZ, Diff) %>%
    arrange(pad_MZ) %>%
    mutate_if(is.numeric, function(x) round(x, 3))

Family,Genus,r_MZ,pad_MZ,r_DZ,pad_DZ,Diff
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ruminococcaceae,Faecalibacterium,0.272,0.051,-0.036,0.993,0.308
Methanobacteriaceae,Methanobrevibacter,0.190,0.150,0.101,0.883,0.089
Christensenellaceae,Christensenella,0.160,0.204,-0.106,0.993,0.267


In [38]:
# Compare mean iccs between MZ and DZ
icc_table %>% select(starts_with("r_")) %>% 
    gather() %>%
    group_by(key) %>%
    summarise(mean = mean(value))

# Mann-Whitney-Wilcoxon test
# Alternative hypothesis:  r_MZ - r_DZ > 0
# i.e r_MZ > r_DZ
U_ICC = icc_table %>% 
    select(starts_with("r_")) %>% 
    gather() %>%
    mutate(key = factor(key, levels = c("r_MZ", "r_DZ"))) %>%
    wilcox.test(value ~ key, data = ., alternative = "greater")

U_ICC

key,mean
<chr>,<dbl>
r_DZ,-0.0008392475
r_MZ,0.1132181043



	Wilcoxon rank sum test with continuity correction

data:  value by key
W = 114560, p-value < 2.2e-16
alternative hypothesis: true location shift is greater than 0


In [39]:
# Permute value
permuted_stat = c()
n_permut = 1000
for (i in 1:n_permut){
    long_ICC = icc_table %>% 
        select(starts_with("r_")) %>% 
        gather()
    
    permuted_zygosity = long_ICC %>%
        pull(key) %>%
        sample(., replace = F) %>%
        mutate(long_ICC, key = .)

    # Mann-Whitney-Wilcoxon test
    U_ICC_per = permuted_zygosity %>%
        mutate(key = factor(key, levels = c("r_MZ", "r_DZ"))) %>%
        wilcox.test(value ~ key, data = ., alternative = "greater")
    permuted_stat = c(permuted_stat, U_ICC_per$statistic)
}

In [40]:
# Empirical p-value
(permuted_stat > U_ICC$statistic) %>% sum/n_permut

[1] 0

# Summary of Methanogens in TUK

In [41]:
# Relative abundance table at genus level
Relabund_Genus = TUK_relabund_genus %>% 
    phyloseq2df(otu_table) 

In [42]:
# Methanogens
Methanogens = c("Methanomassiliicoccus", 
                "Candidatus Methanomethylophilus", 
                "Methanobrevibacter", 
                "Methanosphaera", 
                "Methanoculleus")

Methanogens_Genera = TUK_count_genus %>% 
    phyloseq2df(tax_table) %>% 
    filter(Genus %in% Methanogens) %>%
    select(Genus, OTU)

Methanogens_Genera

Genus,OTU
<chr>,<chr>
Methanobrevibacter,TID2173
Methanosphaera,TID2317
Methanoculleus,TID1262903
Candidatus Methanomethylophilus,TID1769296
Methanomassiliicoccus,TID1406512


In [43]:
# Obtain relabund and presence tables
Methanogens_relabund = Methanogens_Genera %>%
    select(-Genus) %>%
    left_join(., Relabund_Genus) %>%
    gather(Sample, Abundance, -OTU) %>%
    spread(OTU, Abundance) 

Methanogens_pa = Methanogens_relabund %>%
    column_to_rownames("Sample") %>%
    vegan::decostand(method = "pa")

Joining, by = "OTU"


In [44]:
Methanogens_mean = Methanogens_relabund %>%
    summarize_if(is.numeric, function(x) mean(x) * 100) %>%
    mutate(Var = "Mean")

Methanogens_sd = Methanogens_relabund %>%
    summarize_if(is.numeric, function(x) sd(x) * 100) %>%
    mutate(Var = "SD")
                 
Methanogens_prevalence = Methanogens_pa %>%
    summarize_if(is.numeric, function(x) (sum(x)/length(x)) * 100) %>%
    mutate(Var = "Prevalence")
                 
bind_rows(Methanogens_mean, Methanogens_sd, Methanogens_prevalence) %>%
    gather(OTU, Value, -Var) %>% 
    full_join(., Methanogens_Genera) %>%
    spread(Var, Value) %>%
    select(OTU, Genus, Mean, SD, Prevalence) %>%
    arrange(-Mean)

Joining, by = "OTU"


OTU,Genus,Mean,SD,Prevalence
<chr>,<chr>,<dbl>,<dbl>,<dbl>
TID2173,Methanobrevibacter,1.133893e+00,2.7908098830,81.979695
TID2317,Methanosphaera,2.520626e-03,0.0166273858,16.497462
TID1406512,Methanomassiliicoccus,1.632147e-03,0.0181923554,9.390863
TID1769296,Candidatus Methanomethylophilus,8.010035e-05,0.0007309261,1.522843
TID1262903,Methanoculleus,3.736645e-05,0.0002782541,2.538071


# Session Info

In [45]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.4 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/UpSetR_2/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] caret_6.0-84         lattice_0.20-38      irr_0.84.1          
 [4] lpSolve_5.6.15       LeyLabRMisc_0.1.5    cowplot_1.0.0       
 [7] forcats_0.4.0        stringr_1.4.0        dplyr_0.8.3         
[10] purrr_0.3.2          readr_1.3.1          tidyr_0.8.3         
[13] ti